# Aggregate the WECC 2032 ADS GridView Load Data for the TPL-008-01 Weather Zones


In [1]:
# Start by importing the packages we need:
import os
import glob
import datetime
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Suppress Future Warnings


In [2]:
warnings.simplefilter(action='ignore', category=FutureWarning)


## Set the Directory Structure

In [3]:
# Identify the data input and image output directory:
ads_load_data_dir =  '/Users/burl878/Documents/Code/code_repos/gdo_climate_toolsuite_ads2032_loads/data/'
ba_mapping_dir =  '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2026_tbd/data/'
data_output_dir = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2026_tbd/data/load_data/'


## Create a Function to Aggregate the 2032 WECC ADS GridView Data for the TPL-008-1 Weather Zones


In [4]:
def aggregate_gridview_load_data(start_year: int, end_year: int, ba_mapping_dir: str, ads_load_data_dir: str, data_output_dir: str):

    # Loop over the years of TELL load data:
    for year in range(start_year, end_year, 1):
        # Read in the raw data .csv file for the TELL weather year:
        gv_df = pd.read_csv((ads_load_data_dir + 'TELL_Loads/TELL_Loads_2032_Based_on_' + str(year) + '_Weather.csv'))

        # Subset to just hourly demand by BA, drop the index column, and convert the values to floats:
        gv_df = gv_df[1:8785]
        del gv_df["Index"]
        gv_df = gv_df.astype(np.float64)

        # Strip the unecessary bits from the column names:
        gv_df.columns = gv_df.columns.str.replace("_2032.dat", "")
        gv_df.columns = gv_df.columns.str.replace("Load_", "")

        # Delete the BAs in Canada and Mexico and those with zero loads:
        del gv_df['AESO'], gv_df['BCHA'], gv_df['CFE'], gv_df['TH_Malin'], gv_df['TH_Mead'], gv_df['TH_PV']
        
        # Sum the data across BAs in each NERC region:
        gv_df['WECC'] = gv_df.sum(axis=1).round(2)
        gv_df['CA'] = (gv_df['BANC'] + gv_df['CIPB'] + gv_df['CIPV'] + gv_df['CISC'] + gv_df['CISD'] + gv_df['IID'] + gv_df['LDWP'] + gv_df['TIDC']).round(2)
        gv_df['GB'] = (gv_df['IPFE'] + gv_df['IPMV'] + gv_df['IPTV'] + gv_df['NEVP'] + gv_df['PAID'] + gv_df['PAUT'] + gv_df['SPPC'] + gv_df['VEA']).round(2)
        gv_df['PNW'] = (gv_df['AVA'] + gv_df['BPAT'] + gv_df['CHPD'] + gv_df['DOPD'] + gv_df['GCPD'] + gv_df['PACW'] + gv_df['PGE'] + gv_df['PSEI'] + gv_df['SCL'] + gv_df['TPWR']).round(2)
        gv_df['RM'] = (gv_df['NWMT'] + gv_df['PAWY'] + gv_df['PSCO'] + gv_df['WACM'] + gv_df['WAUW']).round(2)
        gv_df['SW'] = (gv_df['AZPS'] + gv_df['EPE'] + gv_df['PNM'] + gv_df['SRP'] + gv_df['TEPC'] + gv_df['WALC']).round(2)
        
        # Make a copy of just the WECC data:
        wecc_df = gv_df[['WECC','CA', 'GB', 'PNW', 'RM', 'SW']].copy()

        # Add in columns with the year and hour of the year:
        wecc_df['Year'] = year
        wecc_df['Hour_of_Year'] = range(len(wecc_df))

        # Compute the actual date:
        wecc_df['Time_UTC'] = pd.Timestamp((str(year) + '-01-01')) + pd.to_timedelta(wecc_df.Hour_of_Year, unit='H')

        # Set date parameters as datetime variables:
        wecc_df['Time_UTC'] = pd.to_datetime(wecc_df['Time_UTC'])

        # Only keep the columns we need:
        temp_df = wecc_df[['Time_UTC','WECC', 'CA', 'GB', 'PNW', 'RM', 'SW']].copy()

        # Aggregate the output into a new dataframe:
        if year == start_year:
           output_df = temp_df
        else:
           output_df = pd.concat([output_df, temp_df])

        # Clean up and move to the next year:
        del gv_df, wecc_df, temp_df

    # Replace zeros with NaN and reset the index value:
    output_df.replace(0, np.nan, inplace=True)
    output_df.reset_index(inplace=True, drop=True)
    
    # Set the output filename:
    output_filename = ('WECC_Hourly_Loads_' + str(start_year) + '_to_' + str(end_year) + '.csv')
        
    # Write out the dataframe to a .csv file:
    output_df.to_csv((os.path.join(data_output_dir, output_filename)), sep=',', index=False)
    
    # Return the output dataframe:
    return output_df


In [5]:
# Execute the function:
output_df = aggregate_gridview_load_data(start_year = 1980,
                                         end_year = 2025, 
                                         ba_mapping_dir = ba_mapping_dir,
                                         ads_load_data_dir = ads_load_data_dir,
                                         data_output_dir = data_output_dir)

output_df


,Time_UTC,WECC,CA,GB,PNW,RM,SW
0,1980-01-01 00:00:00,95949.05,34056.59,11788.15,22629.90,12894.17,14580.24
1,1980-01-01 01:00:00,100408.36,35669.03,12185.68,23480.00,13270.93,15802.71
2,1980-01-01 02:00:00,102709.84,36599.53,12392.81,23855.15,13557.77,16304.58
3,1980-01-01 03:00:00,104593.05,37876.25,12586.23,24162.33,13400.30,16567.95
4,1980-01-01 04:00:00,104588.55,38507.77,12557.12,24018.17,13054.24,16451.25
...,...,...,...,...,...,...,...
395275,2024-12-31 19:00:00,97933.90,33235.87,12232.84,25811.46,13474.88,13178.85
395276,2024-12-31 20:00:00,97279.37,33197.71,12147.71,25500.93,13390.90,13042.12
395277,2024-12-31 21:00:00,97107.29,33381.65,12071.46,25230.91,13337.48,13085.80
395278,2024-12-31 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN


## Create a Function to Calculate the Maximum Load by Day


In [6]:
def process_daily_data(start_year: int, end_year: int, data_output_dir: str):

    # Read in data processed using the function created above:
    load_df = pd.read_csv((data_output_dir + 'WECC_Hourly_Loads_' + str(start_year) + '_to_' + str(end_year) + '.csv'))

    # Set the time as datetime variable:
    load_df['Time_UTC'] = pd.to_datetime(load_df['Time_UTC'])

    # Extract the day from the datetime variable:
    load_df['Date'] = load_df['Time_UTC'].dt.date

    # Compute the daily max for each region:
    load_df['WECC_Max'] = load_df.groupby('Date')['WECC'].transform('max').round(2)
    load_df['CA_Max'] = load_df.groupby('Date')['CA'].transform('max').round(2)
    load_df['GB_Max'] = load_df.groupby('Date')['GB'].transform('max').round(2)
    load_df['PNW_Max'] = load_df.groupby('Date')['PNW'].transform('max').round(2)
    load_df['RM_Max'] = load_df.groupby('Date')['RM'].transform('max').round(2)
    load_df['SW_Max'] = load_df.groupby('Date')['SW'].transform('max').round(2)
        
    # Subset and reorder the columns, drop duplicates, and sort chronologically:
    output_df = load_df[['Date', 'WECC_Max', 'CA_Max', 'GB_Max', 'PNW_Max', 'RM_Max', 'SW_Max']]
    output_df = output_df.drop_duplicates()
    output_df = output_df.sort_values('Date')

    # Rename the columns for simplicity:
    output_df.rename(columns={'WECC_Max': 'WECC', 'CA_Max': 'CA', 'GB_Max': 'GB', 'PNW_Max': 'PNW', 'RM_Max': 'RM', 'SW_Max': 'SW'}, inplace=True)

    # Replace zeros with NaN and reset the index value:
    output_df.replace(0, np.nan, inplace=True)
    output_df.reset_index(inplace=True, drop=True)
    
    # Set the output filename:
    output_filename = ('WECC_Daily_Peak_' + str(start_year) + '_to_' + str(end_year) + '.csv')
        
    # Write out the dataframe to a .csv file:
    output_df.to_csv((os.path.join(data_output_dir, output_filename)), sep=',', index=False)
    
    return output_df


In [7]:
# Execute the function:
output_df = process_daily_data(start_year = 1980,
                               end_year = 2025, 
                               data_output_dir = data_output_dir)

output_df


,Date,WECC,CA,GB,PNW,RM,SW
0,1980-01-01,104593.05,38507.77,12586.23,24340.21,13557.77,16567.95
1,1980-01-02,108800.98,40273.65,13204.76,26062.70,14038.98,17551.85
2,1980-01-03,110798.56,40687.43,13354.41,27249.92,14512.60,18235.76
3,1980-01-04,112635.19,41109.49,13245.75,28093.94,14386.77,18231.21
4,1980-01-05,107661.54,38730.30,12999.35,26221.58,13957.82,16595.96
...,...,...,...,...,...,...,...
16432,2024-12-27,106513.83,38511.18,12841.34,25314.94,13887.43,16086.06
16433,2024-12-28,102396.05,36522.66,12499.18,24410.22,13568.76,15441.15
16434,2024-12-29,101220.76,36424.02,12395.83,23933.70,13268.80,15201.56
16435,2024-12-30,106345.69,38332.64,12507.28,26175.97,13527.27,15859.05
